### Test steps of HMM analysis - before bringing together in module.
Eventually will contain unit tests 
(and code that can transform to unit test)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import socket as socket
import os as os
import sys as sys
import multiprocessing as mp
import h5py
import allel
import itertools as it

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM O2 Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/hapBLOCK/"  # The Path on Harvard Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
sys.path.append("./python3/") 
from emission import load_emission_model
from transition import load_transition_model

print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

compute-a-16-54.o2.rc.hms.harvard.edu
HSM O2 Computational partition detected.
/n/groups/reich/hringbauer/git/hapBLOCK
CPU Count: 32


# Create some test data

In [2]:
### Test data for biallelic case
ht1 = np.array([[0,1],[1,0],[0,1],[0.1,0.9]])
ht2 = np.array([[0,1],[1,0],[1,0],[0.5,0.5]])
ht3 = np.array([[0,1],[1,0],[1,0],[0.5,0.5]])
ht4 = np.array([[0,1],[1,0],[1,0],[0.5,0.5]])
hts = np.stack((ht1, ht2))
hts2 = np.stack((ht1,ht2,ht3,ht4))
p = np.array([0.9,0.8,0.7,0.5])

# Test Emission Module

In [3]:
e = load_emission_model(e_model="haploid_gl")
res =e.give_emission_matrix(hts2, p)

In [34]:
res_exp = np.array([[0.6561, 0.0016, 0.0189, 0.0625],
               [0.729 , 0.008 , 0.    , 0.0625],
               [0.729 , 0.008 , 0.    , 0.0625],
               [0.729 , 0.008 , 0.063 , 0.0625],
               [0.729 , 0.008 , 0.063 , 0.0625]])
assert(np.isclose(res,res_exp).all())
r_vec = np.linspace(0,0.001,101)[1:]

# Test Transition Module

In [5]:
t = load_transition_model(t_model="standard")

In [36]:
t.set_params(ibd_in = 0.0005, ibd_out = 0.001, ibd_jump = 0.05, recalculate=False)
t_mat = t.calc_transition_rate(submat33=False)
assert(np.all(np.isclose(np.sum(t_mat, axis=1),0))) ### Sanity Check

In [35]:
t = load_transition_model(t_model="standard")
t.set_params(ibd_in = 0.0005, ibd_out = 0.001, ibd_jump = 0.05, recalculate=False)
t_full = t.full_transition_matrix(r_vec,t=t_mat,n=4)
np.shape(t_full)

Reference Number: 4


(100, 3, 3)

### Test HMM Module